In [ ]:
# 安装必要
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import numpy as np

# 配置参数
MODEL_NAME = "bert-base-chinese"
MAX_LENGTH = 128
NUM_EPOCHS = 3
BATCH_SIZE = 16


dataset = load_dataset("clue/ner", "msra")

# 查看数据集结构
print(dataset)

# 创建标签映射
id2label = {
    0: "O",
    1: "B-PER",
    2: "I-PER",
    3: "B-ORG",
    4: "I-ORG",
    5: "B-LOC",
    6: "I-LOC",
    7: "B-TIME",
    8: "I-TIME"
}
label2id = {v:k for k,v in id2label.items()}

# 数据预处理
def process_data(examples):
    # 合并字符级标注
    chars = examples["tokens"]
    labels = examples["ner_tags"]
    
    new_labels = []
    for i, label in enumerate(labels):
        word_ids = examples.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
                
        new_labels.append(label_ids)
    
    return {"input_ids": tokenizer(chars, truncation=True, padding="max_length", max_length=MAX_LENGTH),
            "labels": new_labels}

# 初始化分词器
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 处理训练集和验证集
tokenized_datasets = dataset.map(process_data, batched=True)
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))  # 示例使用部分数据
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(200))

# 转换为PyTorch格式
train_dataset.set_format("torch")
eval_dataset.set_format("torch")

# 初始化模型
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)

# 训练参数
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="no",
    load_best_model_at_end=True
)

# 数据收集器
data_collator = DataCollatorForTokenClassification(tokenizer)

# 初始化Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

# 开始训练
trainer.train()

# 推理函数
def predict(text):
    tokens = tokenizer.tokenize(text)
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LENGTH)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    predictions = torch.argmax(outputs.logits, dim=-1).squeeze().tolist()
    
    # 解码预测结果
    entities = []
    current_entity = ""
    current_label = ""
    
    for token, pred in zip(tokens, predictions):
        if pred == 0:  # O标签
            if current_entity:
                entities.append({"entity": current_label, "content": current_entity})
                current_entity = ""
                current_label = ""
        else:
            label = id2label[pred]
            if label.startswith("B-"):
                if current_entity:
                    entities.append({"entity": current_label, "content": current_entity})
                current_entity = token
                current_label = label[2:]
            elif label.startswith("I-"):
                current_entity += token
    
    if current_entity:
        entities.append({"entity": current_label, "content": current_entity})
    
    return entities

# 测试预测
test_text = "双方确定了今后发展中美关系的指导方针。"
print(predict(test_text))
# 示例输出：[{'entity': 'ORG', 'content': '中'}, {'entity': 'ORG', 'content': '美'}]

# 保存模型
model.save_pretrained("./ner_model")
tokenizer.save_pretrained("./ner_model")